In [1]:
#imports
import pandas as pd
import numpy as np
import os
import librosa
import librosa.display
import matplotlib.pyplot as plt
import time
from scipy.io import wavfile as wav
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
from sklearn import metrics 
#keras
from keras.utils import to_categorical
from keras.models import Sequential
from keras.layers import Dense, Dropout, Activation, Flatten
from keras.layers import Convolution2D, MaxPooling2D
from keras.optimizers import Adam
from keras.utils import np_utils

Using TensorFlow backend.


In [2]:
#Settings
audio_dataset_path = "C:\\Users\\Filip\\Desktop\\Jupyter\\wav_dataset"
test_file_on = audio_dataset_path+"\\"+"on"+"\\"+"3cc595de_nohash_1.wav"
test_file_down = audio_dataset_path+"\\"+"down"+"\\"+"b87bdb22_nohash_1.wav"
test_file_right = audio_dataset_path+"\\"+"right"+"\\"+"2aca1e72_nohash_1.wav"
class_label = ["down","go","left","on","right","stop","up"]
wav_sample_rate = 16000
num_mfcc = 40
num_files = 2000
num_epochs = 100
num_batch = 32

In [3]:
def get_spectrogram(file_name):
    try:
        audio, sample_rate = librosa.load(file_name, res_type="kaiser_fast")
        mfcc = librosa.feature.mfcc(y=audio, sr=sample_rate, n_mfcc = num_mfcc)
        scaled = np.mean(mfcc.T, axis=0)
    except Except as e:
        print("Error with file: ", file_name)
        return None, None
    return scaled

In [4]:
def spectro_bot(dataset_path):
    entries = []
    start_time = time.time()
    for dir_name in class_label:
        print(dir_name)
        label_index = class_label.index(dir_name)
        dir_path = dataset_path+"\\"+dir_name
        i = 0
        for file_name in os.listdir(dir_path):
            file_path = dir_path+"\\"+file_name
            data = get_spectrogram(file_path)
            entries.append([data, label_index])
            i=i+1
            if (i==num_files):
                break
    entries_data_frame = pd.DataFrame(entries, columns=["entries", "label"])
    entries_data_frame = entries_data_frame.sample(frac=1).reset_index(drop=True)
    finish_time = time.time()
    print("Finished processing {} files in {} seconds".
          format(len(entries_data_frame), finish_time-start_time))
    return entries_data_frame

In [5]:
data_frame = spectro_bot(audio_dataset_path)
# Convert features and corresponding classification labels into numpy arrays
X = np.array(data_frame.entries.tolist())
y = np.array(data_frame.label.tolist())

# Encode the classification labels
le = LabelEncoder()
yy = to_categorical(le.fit_transform(y))

x_train, x_test, y_train, y_test = train_test_split(X, yy,
                                                    test_size=0.25,
                                                    random_state = 42)



down
go
left
on
right
stop
up
Finished processing 14000 files in 203.51063895225525 seconds


In [6]:
num_labels = yy.shape[1]
#=====================MODEL===========================
model = Sequential()
#input layer
model.add(Dense(256, input_shape=(num_mfcc,)))
model.add(Activation('relu'))
model.add(Dropout(0.5))
#second layer
model.add(Dense(256))
model.add(Activation('relu'))
model.add(Dropout(0.5))
#output layer
model.add(Dense(num_labels))
model.add(Activation('softmax'))
#====================================================

In [7]:
#compile
model.compile(optimizer='adam', 
              loss='categorical_crossentropy',
              metrics=['accuracy'])
model.summary()
# Calculate pre-training accuracy 
score = model.evaluate(x_test, y_test, verbose=0)
accuracy = 100*score[1]

print("Pre-training accuracy: %.4f%%" % accuracy)

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_1 (Dense)              (None, 256)               10496     
_________________________________________________________________
activation_1 (Activation)    (None, 256)               0         
_________________________________________________________________
dropout_1 (Dropout)          (None, 256)               0         
_________________________________________________________________
dense_2 (Dense)              (None, 256)               65792     
_________________________________________________________________
activation_2 (Activation)    (None, 256)               0         
_________________________________________________________________
dropout_2 (Dropout)          (None, 256)               0         
_________________________________________________________________
dense_3 (Dense)              (None, 7)                

In [8]:
#train
start_time = time.time()
model.fit(x_train, y_train, 
          batch_size=num_batch, 
          epochs=num_epochs, 
          validation_data=(x_test, y_test), 
          verbose=1)
finish_time = time.time()
print("Training finished in {} seconds".format(finish_time-start_time))

Train on 10500 samples, validate on 3500 samples
Epoch 1/100
10500/10500 [==============================] - 1s 89us/step - loss: 8.0446 - accuracy: 0.1548 - val_loss: 1.9460 - val_accuracy: 0.1446
Epoch 2/100
10500/10500 [==============================] - 1s 75us/step - loss: 2.0001 - accuracy: 0.1398 - val_loss: 1.9461 - val_accuracy: 0.1383
Epoch 3/100
10500/10500 [==============================] - 1s 71us/step - loss: 1.9632 - accuracy: 0.1417 - val_loss: 1.9462 - val_accuracy: 0.1383
Epoch 4/100
10500/10500 [==============================] - 1s 73us/step - loss: 1.9541 - accuracy: 0.1442 - val_loss: 1.9467 - val_accuracy: 0.1383
Epoch 5/100
10500/10500 [==============================] - 1s 89us/step - loss: 1.9498 - accuracy: 0.1435 - val_loss: 1.9467 - val_accuracy: 0.1386
Epoch 6/100
10500/10500 [==============================] - 1s 73us/step - loss: 1.9523 - accuracy: 0.1427 - val_loss: 1.9461 - val_accuracy: 0.1386
Epoch 7/100
10500/10500 [==============================] - 1s 1

10500/10500 [==============================] - 1s 79us/step - loss: 1.9486 - accuracy: 0.1429 - val_loss: 1.9461 - val_accuracy: 0.1380
Epoch 56/100
10500/10500 [==============================] - 1s 71us/step - loss: 1.9461 - accuracy: 0.1400 - val_loss: 1.9461 - val_accuracy: 0.1386
Epoch 57/100
10500/10500 [==============================] - 1s 73us/step - loss: 1.9463 - accuracy: 0.1416 - val_loss: 1.9461 - val_accuracy: 0.1386
Epoch 58/100
10500/10500 [==============================] - 1s 73us/step - loss: 1.9468 - accuracy: 0.1384 - val_loss: 1.9462 - val_accuracy: 0.1380
Epoch 59/100
10500/10500 [==============================] - 1s 70us/step - loss: 1.9462 - accuracy: 0.1446 - val_loss: 1.9462 - val_accuracy: 0.1386
Epoch 60/100
10500/10500 [==============================] - 1s 70us/step - loss: 1.9463 - accuracy: 0.1384 - val_loss: 1.9462 - val_accuracy: 0.1386
Epoch 61/100
10500/10500 [==============================] - 1s 89us/step - loss: 1.9464 - accuracy: 0.1393 - val_loss: 

In [9]:
# Evaluating the model on the training and testing set
score = model.evaluate(x_train, y_train, verbose=0)
print("Training Accuracy: ", score[1])

score = model.evaluate(x_test, y_test, verbose=0)
print("Testing Accuracy: ", score[1])

Training Accuracy:  0.14428570866584778
Testing Accuracy:  0.13857142627239227


In [10]:
def extract_feature(file_name):
   
    try:
        audio_data, sample_rate = librosa.load(file_name, res_type='kaiser_fast') 
        mfccs = librosa.feature.mfcc(y=audio_data, sr=sample_rate, n_mfcc=40)
        mfccsscaled = np.mean(mfccs.T,axis=0)
        
    except Exception as e:
        print("Error encountered while parsing file: ", file)
        return None, None

    return np.array([mfccsscaled])

In [11]:
def print_prediction(file_name,model):
    prediction_feature = extract_feature(file_name)
    predicted_vector = model.predict_classes(prediction_feature)
    predicted_class = le.inverse_transform(predicted_vector) 
    print("The predicted class is:", class_label[predicted_class[0]], '\n') 
    predicted_proba_vector = model.predict_proba(prediction_feature) 
    predicted_proba = predicted_proba_vector[0]
    for i in range(len(predicted_proba)): 
        category = le.inverse_transform(np.array([i]))
        print(category[0], "\t\t : ", format(predicted_proba[i], '.32f') )


In [12]:
print_prediction(test_file_on, model)

The predicted class is: stop 

0 		 :  0.14084726572036743164062500000000
1 		 :  0.14408060908317565917968750000000
2 		 :  0.14361627399921417236328125000000
3 		 :  0.14359337091445922851562500000000
4 		 :  0.14173272252082824707031250000000
5 		 :  0.14410583674907684326171875000000
6 		 :  0.14202395081520080566406250000000
